# Retrieving the Roemmich-Gilson ARGO based T/S Climatology

In [ ]:
import xarray as xr
from pathlib import Path
from glob import glob
import requests
import warnings
import subprocess as sp
import seawater
import pandas as pd

warnings.filterwarnings("ignore")

def url_retrieve(url: str, outfile: Path):
    R = requests.get(url, allow_redirects=True, verify=False)
    if R.status_code != 200:
        raise ConnectionError('could not download {}\nerror code: {}'.format(url, R.status_code))

    outfile.write_bytes(R.content)

In [ ]:
urlroot = "https://sio-argo.ucsd.edu/gilson/argo_climatology/"

## Download the 2004-2018 climatology

In [ ]:
url_retrieve(f"{urlroot}/RG_ArgoClim_Temperature_2019.nc.gz", Path("./RG_ArgoClim_Temperature_2019.nc.gz"))
url_retrieve(f"{urlroot}/RG_ArgoClim_Salinity_2019.nc.gz", Path("./RG_ArgoClim_Salinity_2019.nc.gz"))

## Download the extension

In [ ]:
files_ext = []
lastyear = 2021
for year in range(2019, lastyear+1):
    for month in range(1, 12+1):
        print(f"retrieving RG_ArgoClim_{year}{month:02g}_2019.nc.gz")
        files_ext.append(f"RG_ArgoClim_{year}{month:02g}_2019.nc")
        url_retrieve(f"{urlroot}/RG_ArgoClim_{year}{month:02g}_2019.nc.gz", Path(f"./RG_ArgoClim_{year}{month:02g}_2019.nc.gz"))

### Decompress files

In [ ]:
for f in glob("*.gz"):
    sp.check_call(f"gunzip --keep --force {f}", shell=True)

## Merge all into a single file

In [ ]:
ds_clim = xr.open_mfdataset(["RG_ArgoClim_Temperature_2019.nc", "RG_ArgoClim_Salinity_2019.nc"], decode_times=False, chunks={"TIME": 12})

In [ ]:
ds_extension = xr.open_mfdataset(files_ext, decode_times=False, chunks={"TIME": 1})

We need to first concatenate time-varying variables then add back the invariant fields:

In [ ]:
invariants = ["ARGO_SALINITY_MEAN", "ARGO_TEMPERATURE_MEAN", "BATHYMETRY_MASK", "MAPPING_MASK"]

ds = xr.concat([ds_clim.drop_vars(invariants), ds_extension], dim="TIME")

for var in invariants:
    ds[var] = ds_clim[var]

In [ ]:
ds.to_netcdf(f"RG_ArgoClim_Extended_200401-{lastyear}12.nc", format="NETCDF3_64BIT")

## Create Annual and Monthly clim for salt and pot_temp

In [ ]:
# compute potential temperature
ptemp = xr.apply_ufunc(seawater.eos80.ptmp,
                       ds["ARGO_SALINITY_MEAN"] + ds["ARGO_SALINITY_ANOMALY"],
                       ds["ARGO_TEMPERATURE_MEAN"] + ds["ARGO_TEMPERATURE_ANOMALY"],
                       ds["PRESSURE"],
                       dask='parallelized',
                       output_dtypes=[ds["ARGO_TEMPERATURE_MEAN"].dtype])

# reconstruct salinity (mean + anomaly)
salt = ds["ARGO_SALINITY_MEAN"] + ds["ARGO_SALINITY_ANOMALY"]

In [ ]:
# create a more flexible time axis
from datetime import timedelta
delta = timedelta(days=15)
time = pd.date_range(start='2004-01', periods=len(ds["TIME"]), freq="1MS") + delta

In [ ]:
# replace time in arrays
ptemp["TIME"] = time
salt["TIME"] = time

In [ ]:
# Annual climatology
ptemp_annual = ptemp.mean(dim="TIME")
salt_annual = salt.mean(dim="TIME")

In [ ]:
# Monthly climatology
ptemp_monthly = ptemp.groupby(ptemp.TIME.dt.month).mean(dim="TIME")
salt_monthly = salt.groupby(salt.TIME.dt.month).mean(dim="TIME")

In [ ]:
# create a depth array
import numpy as np
pressure3d = np.broadcast_to(ds["PRESSURE"].values[:, np.newaxis, np.newaxis],
                                                   (len(ds["PRESSURE"]),
                                                    len(ds["LATITUDE"]),
                                                    len(ds["LONGITUDE"])))
latitude3d = np.broadcast_to(ds["LATITUDE"].values[np.newaxis, :, np.newaxis],
                                                   (len(ds["PRESSURE"]),
                                                    len(ds["LATITUDE"]),
                                                    len(ds["LONGITUDE"])))

depth3d = seawater.eos80.dpth(pressure3d, latitude3d)

In [ ]:
annual_clim = xr.Dataset()
annual_clim["ptemp"] = ptemp_annual.fillna(1.e+20)
annual_clim["salt"] = salt_annual.fillna(1.e+20)
annual_clim["depth"] = xr.DataArray(depth3d, dims=("PRESSURE", "LATITUDE", "LONGITUDE"))

annual_clim = annual_clim.rename({"LONGITUDE": "lon", 
                                  "LATITUDE": "lat",
                                  "PRESSURE": "pres"})


In [ ]:
monthly_clim = xr.Dataset()
monthly_clim["ptemp"] = ptemp_monthly.fillna(1.e+20)
monthly_clim["salt"] = salt_monthly.fillna(1.e+20)
monthly_clim["depth"] = xr.DataArray(depth3d, dims=("PRESSURE", "LATITUDE", "LONGITUDE"))

monthly_clim = monthly_clim.rename({"LONGITUDE": "lon", 
                                    "LATITUDE": "lat",
                                    "PRESSURE": "pres"})

monthly_clim = monthly_clim.transpose(*("month", "pres", "lat", "lon"))

In [ ]:
annual_clim

In [ ]:
monthly_clim

In [ ]:
annual_clim.to_netcdf(f"RG_ArgoClim_Extended_200401-{lastyear}12_annual_clim_ptemp+salt.nc",
                      encoding = {"ptemp": {"_FillValue": 1.e+20},
                                  "salt": {"_FillValue": 1.e+20}},
                      format="NETCDF3_64BIT")

In [ ]:
monthly_clim.to_netcdf(f"RG_ArgoClim_Extended_200401-{lastyear}12_monthly_clim_ptemp+salt.nc",
                       encoding = {"ptemp": {"_FillValue": 1.e+20},
                                  "salt": {"_FillValue": 1.e+20}},
                       format="NETCDF3_64BIT", unlimited_dims="month")